In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as st
import statsmodels as sm
import statsmodels.formula.api as smf
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn import metrics
from sklearn import preprocessing
from sklearn import decomposition
from sklearn.preprocessing import scale
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

### Mission 3
Modélisez les données à l'aide d'une régression logistique. Grâce à celle-ci, vous créerez un programme capable d'effectuer une prédiction sur un billet, c'est-à-dire de déterminer s'il s'agit d'un vrai ou d'un faux billet. Pour chaque billet, votre algorithme de classification devra donner la probabilité que le billet soit vrai. Si cette probabilité est supérieure ou égale à 0.5, le billet sera considéré comme vrai. Dans le cas contraire, il sera considéré comme faux.

In [2]:
#Importation du fichier source
data = pd.read_csv(r"C:\Users\elodi\Documents\DATA\Data Analyst\Projets\P6\notes.csv", sep=',')

### 1. Régression logistique

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 170 entries, 0 to 169
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   is_genuine    170 non-null    bool   
 1   diagonal      170 non-null    float64
 2   height_left   170 non-null    float64
 3   height_right  170 non-null    float64
 4   margin_low    170 non-null    float64
 5   margin_up     170 non-null    float64
 6   length        170 non-null    float64
dtypes: bool(1), float64(6)
memory usage: 8.3 KB


In [4]:
#Remplacement des valeurs true/false par 1 et 0
data['is_genuine'].replace([True, False], [1, 0], inplace = True)
display(data)

,is_genuine,diagonal,height_left,height_right,margin_low,margin_up,length
0,1,171.81,104.86,104.95,4.52,2.89,112.83
1,1,171.67,103.74,103.70,4.01,2.87,113.29
2,1,171.83,103.76,103.76,4.40,2.88,113.84
3,1,171.80,103.78,103.65,3.73,3.12,113.63
4,1,172.05,103.70,103.75,5.04,2.27,113.55
...,...,...,...,...,...,...,...
165,0,172.11,104.23,104.45,5.24,3.58,111.78
166,0,173.01,104.59,104.31,5.04,3.05,110.91
167,0,172.47,104.27,104.10,4.88,3.33,110.68
168,0,171.82,103.97,103.88,4.73,3.55,111.87


In [5]:
#Préparation des données
X = data.copy()
X = X.iloc[:, 1:]
y = data.iloc[:, 0]

In [6]:
#Séparation données test et d'entraînement
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [7]:
#Instantiation du modèle
lr = LogisticRegression()
lr.fit(X_train, y_train)

LogisticRegression()

##### Vérification de la colinéarité des variables
A partir du VIF (Variance Inflation Factor), le facteur d'influence de la variable, nous pouvons avoir une idée des éventuels problèmes de colinéarité.

In [8]:
reg_multi = smf.ols('diagonal ~ height_left + height_right + margin_low + margin_up + length', data=data).fit()
variables = reg_multi.model.exog
[variance_inflation_factor(variables, i) for i in np.arange(1,variables.shape[1])]

[2.249831212995871,
 2.5637087947433295,
 2.172802257913991,
 1.6252305296552272,
 2.463836435483419]

# 
Si des variables colinéaires sont de facto fortement corrélées entre elles, deux variables corrélées ne sont pas forcément colinéaires. La régression logistique peut être modélisée sur nos six variables explicatives, il ne semble pas avoir de frein possible, ni colinéarité, il n'y a non plus pas de valeur atypique influente

##### Matrice de confusion (évalutation du modèle)

In [9]:
y_pred = lr.predict(X_test)
cf_matrix = confusion_matrix(y_test, y_pred)
print(cf_matrix)

[[13  0]
 [ 0 21]]


In [10]:
ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues')

ax.set_title('Matrice de confusion\n\n');
ax.set_xlabel('\nValeurs prédites')
ax.set_ylabel('Valeurs actuelles');


ax.xaxis.set_ticklabels(['False','True'])
ax.yaxis.set_ticklabels(['False','True'])

"plt.savefig('matrice_lr.png')
plt.show()

SyntaxError: EOL while scanning string literal (<ipython-input-10-3c6394090c84>, line 11)

##### L'évaluation du modèle peut aussi se faire par la courbe ROC et sa métrique AUC.
La courbe ROC (Receiver Operating Characteristic) représente la sensibilité en fonction de la spécificité pour différents seuils de décision. L'aire sous la courbe ROC, l'AUC (Area Under the ROC ), est une mesure de la qualité de la classification qui varie entre 0.5 dans le pire des cas et 1 dans le meilleur des cas.

In [ ]:
y_prob = lr.predict_proba(X_test)[:,1] 
false_positive_rate, true_positive_rate, thresholds = metrics.roc_curve(y_test, y_prob)

In [ ]:
metrics.auc(false_positive_rate, true_positive_rate)

##### Courbe ROC

In [ ]:
plt.figure(figsize=(10, 8))

plt.plot(false_positive_rate, true_positive_rate)
plt.plot([0, 1], [0, 1], linestyle='--')

plt.xlabel('TFP')
plt.ylabel('TVP')
plt.title('Courbe ROC')

#plt.savefig('roc.png')
plt.show()

##### Autres métriques d'évaluation du modèle 

In [ ]:
print('  - Accuracy:' +str(round(metrics.accuracy_score(y_test, y_pred)*100, 2)), '%')
print('  - Precision:' +str(round(metrics.precision_score(y_test, y_pred)*100, 2)), '%')
print('  - Recall:' +str(round(metrics.recall_score(y_test, y_pred)*100, 2)), '%')
print('  - F1 score:' +str(round(metrics.f1_score(y_test, y_pred)*100, 2)), '%')

# 
Les métriques d'évaluation que nous obtenons sont très satisfaisantes. Après l'évaluation des performances du modèle de prédiction 'lr', ce modèle est retenu pour sa fiablité et sa performance.

### 2. Application sur le fichier test

In [ ]:
df_test = pd.read_csv(r"C:\Users\elodi\Downloads\example (1).csv")
df_test

In [ ]:
#Préparation des données
X = df_test.copy()
X = X.iloc[:, :-1]

In [ ]:
#Utilisation du modèle de prédiction 'lr'
probability = lr.predict_proba(X)[:,1]
probability

In [ ]:
proba = pd.Series(probability.round(3), name='value')

In [ ]:
#Intégration des probabilités dans le jeu de données
df_pred = pd.concat([df_test, proba], axis=1)
df_pred

##### Résultat de la classification prédictive :

In [ ]:
resultat = []
for i in df_pred['value'] >= .5:
        if i is True :
            resultat.append('Vrai Billet')
        else :
            resultat.append('Faux Billet')

df_pred['resultat'] = resultat
df_pred

### 3. Algorithme de classification des billets de banque

In [ ]:
def detecteur(fichier_csv):
    df = pd.read_csv(fichier_csv)
    X = df.copy()
    X = X.iloc[:, :-1]
    y = data.iloc[:, 0]
    
    #Centrage/réduction des données
    std_scale = preprocessing.StandardScaler().fit(X)
    X_scaled = std_scale.transform(X)
     
    #Utilisation du modèle de prédiction 'lr'
    probability = lr.predict_proba(X_scaled)[:, 1]
    
    #Probabilités des billets établies 
    proba = pd.Series(probability.round(3), name='%value')
    
    #Intégration des probabilités dans le jeu de données
    df_final = pd.concat([df, proba], axis=1)
    
   #Résultat 
    resultat = []
    for i in df_final['%value'] >= .5:
        if i is True :
            resultat.append('Vrai Billet')
        else :
            resultat.append('Faux Billet')

    df_final['resultat'] = resultat

    return df_final   

In [ ]:
detecteur(r"C:\Users\elodi\Downloads\example (1).csv")